In [25]:
import warnings
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

warnings.filterwarnings("ignore", category=UserWarning)

In [26]:
import torch
from transformers import BertTokenizerFast, BertForMultipleChoice, Trainer, TrainingArguments, DataCollatorForTokenClassification
from datasets import load_dataset


In [27]:
# Функция подготовки выборок для multiple choice task
def prepare_features(batch):
    # Конструируем prompt для каждого варианта выбора
    choices = [batch['choice1'], batch['choice2']]
    premise = [batch['premise'] + ', ' + batch['question']] * len(choices)

    # Токенизируем с сохранением связи индексов
    inputs = tokenizer(premise, choices, truncation=True, max_length=128, padding="max_length", return_tensors="pt")

    # Добавляем метку правильного ответа
    label = batch.get('label')  # Если label отсутствует, значит тестовая выборка
    if label is not None:
        inputs['labels'] = label
    return inputs

In [28]:
# Загружаем предварительно подготовленный датасет
ds = load_dataset("PARus")

model_name = "DeepPavlov/rubert-base-cased"
# Загружаем предобученную модель
model = BertForMultipleChoice.from_pretrained(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    model = model.to("cuda")
# Подготавливаем токенизатор
tokenizer = BertTokenizerFast.from_pretrained(model_name)

# # Обрабатываем датасеты
for split_name in ['train', 'validation']:
    ds[split_name] = ds[split_name].map(prepare_features,
                                        batched=False,
                                        remove_columns=['premise', 'choice1', 'choice2', 'question', 'label']
                                        )


Some weights of BertForMultipleChoice were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
# Преобразуем объект в PyTorch TensorDataset
ds["train"].set_format(type='torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'labels'])
ds["validation"].set_format(type='torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'labels'])

ds["train"]

Dataset({
    features: ['idx', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 400
})

In [30]:
# Определение метрик точности (accuracy)
from sklearn.metrics import accuracy_score


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = logits.argmax(axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

In [31]:
# Аргументы обучения
training_args = TrainingArguments(
    output_dir='./rubert_finetuned_PARus',
    eval_strategy="epoch",
    save_strategy='epoch',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    num_train_epochs=3,
    warmup_ratio=0.1,
    weight_decay=0.01,
    save_total_limit=1,
    greater_is_better=True,
    disable_tqdm=False,
    load_best_model_at_end=True,
)


# Создаем тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds['train'],
    eval_dataset=ds['validation'],
    compute_metrics=compute_metrics,
)



In [ ]:
# Начинаем обучение
trainer.train()


Epoch,Training Loss,Validation Loss


In [93]:
# Оцениваем модель на валидирующем наборе
trainer.evaluate()


{'eval_loss': 0.9222877621650696,
 'eval_accuracy': 0.64,
 'eval_runtime': 30.9749,
 'eval_samples_per_second': 3.228,
 'eval_steps_per_second': 0.42,
 'epoch': 10.0}

In [94]:
# Сохранение финальной модели
trainer.save_model("./final_model")